3. Сгенерировать последовательности, которые состоят из цифр (от 0 до 9) и задаются следующим образом:
x - последовательность цифр
y1 = x1 
yi = xi + x1
Если yi >= 10 то yi = yi - 10
Научить модель рекуррентной нейронной сети предсказывать yi по xi Использовать: RNN, LSTM, GRU


In [2]:
import numpy as np
import torch
from itertools import permutations
import random
import time
import torch.nn.functional 


Подготовка данных

In [4]:
# функция для перестановки заданного списка
# генерируем последовательность из цифр от 0-9:
def permutation(lst):
 
    # Если lst пуст, то перестановок нет
    if len(lst) == 0:
        return []
 
    # если 1 элемент, то одна перестановка возможна
   
    if len(lst) == 1:
        return [lst]
 
    # Найдите перестановки для lst, если они существую
    # более чем 1 символ
 
    l = [] # пустой список, в котором будет храниться текущая перестановка
 
    # Повторите ввод (lst) и вычислите перестановку
    for i in range(len(lst)):
       m = lst[i]
 
       # Extract lst[i] or m from the list.  remLst is
       # remaining list
       remLst = lst[:i] + lst[i+1:]
 
       # Генерация всех перестановок, где m является первым элементом
       
       for p in permutation(remLst):
           l.append([m] + p)
    return l
 
 
# тест функции, добавим все перестановки в список
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 0]
list = []
for p in permutation(data):
    list.append(p)
    #print (p)

In [5]:
#длина полученного списка
list[:5]

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 0],
 [1, 2, 3, 4, 5, 6, 7, 8, 0, 9],
 [1, 2, 3, 4, 5, 6, 7, 9, 8, 0],
 [1, 2, 3, 4, 5, 6, 7, 9, 0, 8],
 [1, 2, 3, 4, 5, 6, 7, 0, 8, 9]]

In [6]:
len(list)

3628800

In [7]:
#отбираем из списка по индексу кратный 50, так как list слишком большой
Xdata = []
for m in range(0, len(list)):
    if m %50==0:
        Xdata.append(list[m])

In [8]:
len(Xdata)

72576

In [9]:
#функция - прописываем условия для Y
def create_dataset(df):
    set = []
    ydata = []
    set.append(df[0])
    for i in range(1, len(df)):
        set.append(df[i]+df[0])
    for j in range(0, len(set)):
        if set[j] >= 10:
            ydata.append(set[j]-10)
        else:
            ydata.append(set[j]) 
    return ydata

In [10]:
#тест
df = [1, 2, 3, 4, 5, 6, 7, 8, 9, 0]
create_dataset(df)

[1, 3, 4, 5, 6, 7, 8, 9, 0, 1]

In [11]:
#применяем фугкцию для всех элементов списка
Ydata=[]
for k in Xdata:
    Ydata.append(create_dataset(k))

In [12]:
len(Ydata)

72576

In [13]:
Ydata[:5]

[[1, 3, 4, 5, 6, 7, 8, 9, 0, 1],
 [1, 3, 4, 5, 6, 9, 7, 0, 8, 1],
 [1, 3, 4, 5, 6, 1, 7, 0, 8, 9],
 [1, 3, 4, 5, 7, 8, 9, 6, 0, 1],
 [1, 3, 4, 5, 7, 0, 8, 9, 6, 1]]

In [14]:
# делим на train/test
num_trn = int(0.75*len(Xdata))                
X_trn, Y_trn = Xdata[:num_trn], Ydata[:num_trn]
X_test, Y_test = Xdata[num_trn:], Ydata[num_trn:]

In [15]:
len(X_trn)

54432

In [16]:
len(Y_trn)

54432

In [17]:
len(X_test)

18144

In [18]:
len(Y_test)

18144

GRU

In [20]:
class SimpleGRU(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(SimpleGRU, self).__init__()
        self.hidden_size  = hidden_size
        self.num_layers = num_layers
        self.gru = torch.nn.GRU(input_size=10, hidden_size=hidden_size, num_layers=num_layers,batch_first=True)
        self.fc1 = torch.nn.Linear(hidden_size, output_dim)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        out,_ = self.gru(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc1(out)
        return out

In [21]:
input_size = 10
hidden_size = 100
num_layers = 1
output_dim = 10
model = SimpleGRU(input_size, hidden_size, num_layers)
optimizer =torch.optim.Adam(model.parameters())
loss = torch.nn.MSELoss()
#loss = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

Обучаем модель

In [23]:
for epoch in range(20):
    train_loss, train_acc, iter_num = .0, .0, .0
    start_epoch_time = time.time()
    model.train()
    X_trn = torch.tensor(X_trn, dtype = torch.float32).view(len(X_trn), 1, -1)
    Y_trn = torch.tensor(Y_trn, dtype = torch.float32).view(len(Y_trn), -1)
    #print(X_trn.shape)
    #print(Y_trn.shape)
    optimizer.zero_grad() 
    out = model.forward(X_trn) 
    #print(out.shape)
    out = out.view(len(Y_trn), -1)
    #print(out.shape)
    #print(Y_trn.shape)
    l = loss(out, Y_trn)
    train_loss += l.item()
    l.backward()
    optimizer.step()
    print(f"Epoch: {epoch}, loss: {train_loss:.4f}",end=" | ")

Epoch: 0, loss: 28.4531 | 

C:\Users\user\AppData\Local\Temp\ipykernel_10476\2521572634.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_trn = torch.tensor(X_trn, dtype = torch.float32).view(len(X_trn), 1, -1)
C:\Users\user\AppData\Local\Temp\ipykernel_10476\2521572634.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_trn = torch.tensor(Y_trn, dtype = torch.float32).view(len(Y_trn), -1)


Epoch: 1, loss: 27.9757 | Epoch: 2, loss: 27.5040 | Epoch: 3, loss: 27.0364 | Epoch: 4, loss: 26.5709 | Epoch: 5, loss: 26.1054 | Epoch: 6, loss: 25.6379 | Epoch: 7, loss: 25.1668 | Epoch: 8, loss: 24.6907 | Epoch: 9, loss: 24.2085 | Epoch: 10, loss: 23.7194 | Epoch: 11, loss: 23.2226 | Epoch: 12, loss: 22.7178 | Epoch: 13, loss: 22.2047 | Epoch: 14, loss: 21.6836 | Epoch: 15, loss: 21.1548 | Epoch: 16, loss: 20.6188 | Epoch: 17, loss: 20.0766 | Epoch: 18, loss: 19.5293 | Epoch: 19, loss: 18.9782 | 

Проверка на тестовый данных

In [25]:
for epoch in range(50):
    test_loss = .0
    start_epoch_time = time.time()
    model.eval()
    X_test = torch.tensor(X_test, dtype = torch.float32).view(len(X_test), 1, -1)
    Y_test = torch.tensor(Y_test, dtype = torch.float32).view(len(Y_test), -1)
    out = model.forward(X_test) 
    out = out.view(len(Y_test), -1)
    l = loss(out, Y_test)
    test_loss += l.item()
    l.backward()
    optimizer.step()
    print(f"Epoch: {epoch}, loss: {test_loss:.4f}",end=" | ")

Epoch: 0, loss: 19.4472 | Epoch: 1, loss: 18.8837 | 

C:\Users\user\AppData\Local\Temp\ipykernel_10476\3419201947.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test, dtype = torch.float32).view(len(X_test), 1, -1)
C:\Users\user\AppData\Local\Temp\ipykernel_10476\3419201947.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_test = torch.tensor(Y_test, dtype = torch.float32).view(len(Y_test), -1)


Epoch: 2, loss: 18.3139 | Epoch: 3, loss: 17.7401 | Epoch: 4, loss: 17.1620 | Epoch: 5, loss: 16.5801 | Epoch: 6, loss: 15.9956 | Epoch: 7, loss: 15.4110 | Epoch: 8, loss: 14.8295 | Epoch: 9, loss: 14.2545 | Epoch: 10, loss: 13.6898 | Epoch: 11, loss: 13.1393 | Epoch: 12, loss: 12.6068 | Epoch: 13, loss: 12.0963 | Epoch: 14, loss: 11.6112 | Epoch: 15, loss: 11.1551 | Epoch: 16, loss: 10.7314 | Epoch: 17, loss: 10.3430 | Epoch: 18, loss: 9.9928 | Epoch: 19, loss: 9.6832 | Epoch: 20, loss: 9.4163 | Epoch: 21, loss: 9.1940 | Epoch: 22, loss: 9.0175 | Epoch: 23, loss: 8.8877 | Epoch: 24, loss: 8.8048 | Epoch: 25, loss: 8.7685 | Epoch: 26, loss: 8.7782 | Epoch: 27, loss: 8.8326 | Epoch: 28, loss: 8.9300 | Epoch: 29, loss: 9.0684 | Epoch: 30, loss: 9.2455 | Epoch: 31, loss: 9.4585 | Epoch: 32, loss: 9.7046 | Epoch: 33, loss: 9.9805 | Epoch: 34, loss: 10.2823 | Epoch: 35, loss: 10.6060 | Epoch: 36, loss: 10.9472 | Epoch: 37, loss: 11.3011 | Epoch: 38, loss: 11.6624 | Epoch: 39, loss: 12.0257 

Результаты неудовлетворительные по всем моделям (RNN/LSTM/GRU). Возможно некорректные данные для обучения или ошибки в модели